# H-Field Probe Demo 1 (with CPA)

This notebook is designed to work with an H-Field Probe. This can be used to attack devices without needing to modify them with a shunt resistor at all.

## Normal Setup

Perform a normal setup as you otherwise would. Adjust the platform & target as needed. Here we'll attack an XMEGA device on the CW308 UFO board for example. Note that because the UFO boards are missing the decoupling capacitors & have a resistive shunt, you may find they perform *worse* for H-Field probe usage than normal development kits.

Neverless, we're going to give it a shot! We found the XMEGA works better than the STM32F3 target on the UFO board, so we'll do this tutorial with the XMEGA target board.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'TINYAES128C'
num_traces = 50
CHECK_CORR = False

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

In [ ]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)

In [ ]:
cw.program_target(scope, prog, fw_path)

## Adjustment for H-Field Probe

Now time for the H-Field probe. We'll do the following:

1. Attach LNA input to the H-Field probe.
2. Attack LNA output to the ChipWhisperer measure input.
3. Power up the LNA.
4. Crank up the gain on the ChipWhisperer.

Items 1-3 are done in this photo:

<img src="img/hprobe_xmega_setup.jpg" alt="SPA of Power Analysis">

Item 4 is done with the following line of code:

In [ ]:
scope.gain.gain = 70

## Visual Capture Loop

Now adjusting the H-Field probe around, you can find an interesting location. The following loop will run up to 5000 captures, but throws them away. Just stop the loop if you find a good physical location (traces look stable). This might not be super-obvious: you'll see how to run a CPA attack next.

In [ ]:
%run "../Helper_Scripts/plot.ipynb"
plot = real_time_plot(plot_len=3000)

In [ ]:
from tqdm import tnrange
ktp = cw.ktp.Basic()

key, text = ktp.next()

for i in tnrange(5000, desc='Capturing traces'):    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    plot.send(trace)

## Actual Capture Loop & Analysis

Then run the following two blocks - if the results aren't good, try another location!

In [ ]:
import chipwhisperer as cw
import chipwhisperer.analyzer as cwa

project = cw.create_project("projects/hfield_demo.cwp", overwrite = True)

from tqdm import tnrange
ktp = cw.ktp.Basic()

for i in tnrange(250, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    project.traces.append(trace)
    #send every 25th trace
    if i % 25 == 0:
        plot.send(trace)
    
project.save()

attack = cwa.cpa(project, cwa.leakage_models.sbox_output)
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 25)

## Next Steps & Cleanup

You might notice that it's not super-obivuous how strong the signal is. Luckily there are better ways of getting quick feedback on leakage - in particular you can use the TVLA example code to plot in close-to-realtime a "T-Test", which will allow you to pinpoint leakage in a white-box scenario (i.e., you control the text & key).

Hopefully this worked for you. Run the following to close the connection.

In [ ]:
scope.dis()
target.dis()